# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)

/home/workspace
/home/workspace/event_data


In [3]:
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/work

#### Creating list of filepaths to process original event csv data files

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

            
print(len(full_data_rows_list))
#print(full_data_rows_list)

8056


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:

            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:

# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_db 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify_db')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
session.execute("""
create table if not exists sessionOfsong 
(
sessionId int,
itemInSession int,
artist text,
song text,
length float,
primary key (sessionId, itemInSession)
)""") 

                    

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into sessionOfsong (sessionId, itemInSession, artist, song, length)"
        query = query + "values (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "select artist, song, length from sessionOfsong where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [15]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# convert the original data from csv to pandas.DataFrame
file = 'event_datafile_new.csv'
#filepath = '../data/' + file
df = pd.read_csv(file)

session.execute("""
CREATE TABLE IF NOT EXISTS userOfsongs
(
  userId int,
  sessionId int,
  itemInSession int,
  artist text, 
  song text, 
  user text,
  PRIMARY KEY ((userId, sessionId), itemInSession)
)
""")

user_data = df.loc[:, ['artist', 'song', 'firstName', 'lastName', 'userId', 'sessionId', 'itemInSession']]
user_data['user'] = user_data['firstName'] + '.' + user_data['lastName']
user_data = user_data.drop(columns=['firstName', 'lastName'])
#print(user_data)

query2 = """
INSERT INTO userOfsongs (artist, song, userId, sessionId, itemInSession, user)
VALUES (%s, %s, %s, %s, %s, %s)"""

for row in user_data.iterrows():
    data_as_tuple = tuple(row[1].values)
    session.execute(query2, data_as_tuple)
                    

In [16]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

session.execute("""
CREATE TABLE IF NOT EXISTS dataOfsongs
(
  song text, 
  userId int,
  artist text, 
  user text,
  PRIMARY KEY (song, userId)
)
""")

song_data = df.loc[:, ['artist', 'song', 'firstName', 'lastName', 'userId']]
song_data['user'] = song_data['firstName'] + '.' + song_data['lastName']
song_data = song_data.drop(columns=['firstName', 'lastName'])

query3 = """
INSERT INTO dataOfsongs (artist, song, userId, user)
VALUES (%s, %s, %s, %s)"""

for row in song_data.iterrows():
    data_as_tuple = tuple(row[1].values)
    session.execute(query3, data_as_tuple)

                    

InvalidRequest: Error from server: code=2200 [Invalid query] message="Undefined column name userid"

### Drop the tables before closing out the sessions

In [16]:
## TO-DO: Drop the table before closing out the sessions
query = "DROP TABLE sessions"
try:
    session.execute(query)
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table sessions"


In [17]:
## TO-DO: Drop the table before closing out the sessions
query = "DROP TABLE sessions"
try:
    session.execute(query2)
except Exception as e:
    print(e)

<Error from server: code=2000 [Syntax error in CQL query] message="line 3:8 no viable alternative at character '%'">


In [18]:
## TO-DO: Drop the table before closing out the sessions
query = "DROP TABLE sessions"
try:
    session.execute(query3)
except Exception as e:
    print(e)

<Error from server: code=2000 [Syntax error in CQL query] message="line 3:8 no viable alternative at character '%'">


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()